In [5]:
!pip install -q memory_profiler psutil

import time
import random
import tracemalloc
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

random.seed(101)
np.random.seed(101)

%matplotlib inline
plt.rcParams['figure.figsize'] = (9,6)


In [6]:
def measure_time_and_memory(func, *args, repeat=5, warmup=2, **kwargs):
    times = []
    mems = []
    last_result = None

    for _ in range(warmup):
        try:
            func(*args, **kwargs)
        except:
            pass

    for _ in range(repeat):
        tracemalloc.start()
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        t1 = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        times.append(t1 - t0)
        mems.append(peak / (1024*1024))
        last_result = result

    return {
        'time_mean': float(np.mean(times)),
        'time_std': float(np.std(times)),
        'mem_mean_mb': float(np.mean(mems)),
        'mem_std_mb': float(np.std(mems)),
        'result': last_result
    }


In [10]:
# Fibonacci recursive
def fib_recursive_with_depth(n):
    max_depth = 0
    sys.setrecursionlimit(12000)
    def _fib(k, depth=1):
        nonlocal max_depth
        max_depth = max(max_depth, depth)
        if k <= 1:
            return k
        return _fib(k-1, depth+1) + _fib(k-2, depth+1)
    val = _fib(n, 1)
    return {'value': val, 'max_recursion_depth': max_depth}

# Fibonacci DP
def fib_dp(n):
    if n <= 1:
        return {'value': n, 'max_recursion_depth': 1}
    a, b = 0, 1
    for _ in range(2, n+1):
        a, b = b, a+b
    return {'value': b, 'max_recursion_depth': 1}

# Merge Sort
def merge_sort_with_depth(arr):
    max_depth = 0
    def _merge(a, depth=1):
        nonlocal max_depth
        max_depth = max(max_depth, depth)
        if len(a) <= 1:
            return a
        mid = len(a)//2
        left = _merge(a[:mid], depth+1)
        right = _merge(a[mid:], depth+1)
        i = j = 0
        merged = []
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                merged.append(left[i]); i+=1
            else:
                merged.append(right[j]); j+=1
        merged.extend(left[i:]); merged.extend(right[j:])
        return merged
    sorted_arr = _merge(list(arr), 1)
    return {'sorted': sorted_arr, 'max_recursion_depth': max_depth}

# Quick Sort
def quick_sort_with_depth(arr):
    max_depth = 0
    def _quick(a, low, high, depth):
        nonlocal max_depth
        if low >= high:
            max_depth = max(max_depth, depth)
            return
        max_depth = max(max_depth, depth)
        pivot_idx = random.randint(low, high)
        a[pivot_idx], a[high] = a[high], a[pivot_idx]
        pivot = a[high]
        i = low
        for j in range(low, high):
            if a[j] <= pivot:
                a[i], a[j] = a[j], a[i]; i+=1
        a[i], a[high] = a[high], a[i]
        _quick(a, low, i-1, depth+1)
        _quick(a, i+1, high, depth+1)
    a_copy = list(arr)
    _quick(a_copy, 0, len(a_copy)-1, 1)
    return {'sorted': a_copy, 'max_recursion_depth': max_depth}

# Insertion Sort
def insertion_sort(arr):
    a = list(arr)
    for i in range(1, len(a)):
        key = a[i]
        j = i-1
        while j >= 0 and a[j] > key:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = key
    return {'sorted': a}

# Bubble Sort
def bubble_sort(arr):
    a = list(arr)
    n = len(a)
    for i in range(n):
        swapped = False
        for j in range(0, n-i-1):
            if a[j] > a[j+1]:
                a[j], a[j+1] = a[j+1], a[j]
                swapped = True
        if not swapped:
            break
    return {'sorted': a}

# Selection Sort
def selection_sort(arr):
    a = list(arr)
    n = len(a)
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if a[j] < a[min_idx]:
                min_idx = j
        a[i], a[min_idx] = a[min_idx], a[i]
    return {'sorted': a}

# Binary Search
def binary_search(arr, target):
    lo, hi = 0, len(arr)-1
    steps = 0
    while lo <= hi:
        mid = (lo+hi)//2
        steps += 1
        if arr[mid] == target:
            return {'index': mid, 'steps': steps}
        elif arr[mid] < target:
            lo = mid+1
        else:
            hi = mid-1
    return {'index': -1, 'steps': steps}


In [8]:
print('fib_recursive(7):', fib_recursive_with_depth(7))
print('fib_dp(20):', fib_dp(20)['value'])
print('merge_sort example:', merge_sort_with_depth([8,3,7,5])['sorted'])
print('quick_sort example:', quick_sort_with_depth([11,6,14,2,7])['sorted'])
print('insertion sort example:', insertion_sort([10,4,8,2])['sorted'])
print('binary search example:', binary_search([3,6,9,12,15], 12))


fib_recursive(7): {'value': 13, 'max_recursion_depth': 7}
fib_dp(20): 6765
merge_sort example: [3, 5, 7, 8]
quick_sort example: [2, 6, 7, 11, 14]
insertion sort example: [2, 4, 8, 10]
binary search example: {'index': 3, 'steps': 2}
